In [1]:
import sys
sys.path.append('/root/llm-persona-superposition/llm-personal-superposition/src/steer_experiments')
from SAE.Steering import *

set_up()
model_name = "gemma-2b-it"
sae_name = "gemma-2b-it-res-jb"
layer = 12
model, sae, hook_point = load_model(model_name, sae_name, layer)

Device: cuda


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer
blocks.12.hook_resid_post


In [45]:
def create_steering_hook(coeffs, steering_vectors, steering_on):
    torch.set_grad_enabled(False)
    def steering_hook(resid_pre, hook):
        if resid_pre.shape[1] == 1:
            return
        position = resid_pre.shape[1]
        if steering_on:
            for steering_vector, coeff in zip(steering_vectors, coeffs):
                resid_pre[:, :position - 1, :] += coeff * steering_vector
    return steering_hook

def hooked_generate(model, prompt, fwd_hooks=[], seed=None, **kwargs):
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt)
        result = model.generate(
            stop_at_eos=False,  # avoids a bug on MPS
            input=tokenized,
            max_new_tokens=30,
            do_sample=True,
            **kwargs)
    return result

def run_generate(example_prompt, model, layer, coeff, steering_vector, steering_on, sampling_kwargs, seed=None):
    model.reset_hooks()
    steering_hook = create_steering_hook(coeff, steering_vector, steering_on)
    editing_hooks = [(f"blocks.{layer}.hook_resid_post", steering_hook)]
    res = hooked_generate(model, example_prompt, editing_hooks, seed=seed, **sampling_kwargs)
    # Print results, removing the ugly beginning of sequence token
    res_str = model.to_string(res[:, 1:])
    print(("\n\n" + "-" * 80 + "\n\n").join(res_str))

In [21]:
def create_steering_hook(coeffs, steering_vectors, steering_on):
    torch.set_grad_enabled(False)
    def steering_hook(resid_pre, hook):
        if resid_pre.shape[1] == 1:
            return
        position = resid_pre.shape[1]
        if steering_on:
            for steering_vector, coeff in zip(steering_vectors, coeffs):
                resid_pre[:, :position - 1, :] += coeff * steering_vector
    return steering_hook

def hooked_generate(model, prompt, fwd_hooks=[], seed=None, **kwargs):
    if seed is not None:
        torch.manual_seed(seed)
    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt)
        with torch.no_grad():
            outputs = model(tokenized)
        logits = outputs[:, -1, :]  # Logits for the last token
        probabilities = torch.softmax(logits, dim=-1)
    return probabilities

def run_generate(example_prompt, model, layer, coeff, steering_vector, steering_on, sampling_kwargs, seed=None):
    model.reset_hooks()
    steering_hook = create_steering_hook(coeff, steering_vector, steering_on)
    editing_hooks = [(f"blocks.{layer}.hook_resid_post", steering_hook)]
    res = hooked_generate(model, example_prompt, editing_hooks, seed=seed, **sampling_kwargs)


In [8]:
steering_vectors = [sae.W_dec[5361], sae.W_dec[4365]]
coeff = [100, 100]
question_prompt = "give me only one random color and animal."
sampling_kwargs = dict(temperature=0.2, freq_penalty=1.0)
seed = 16

In [20]:
steering_on = False
run_generate(question_prompt, model, layer, coeff, steering_vectors, steering_on, sampling_kwargs,seed)

torch.Size([1, 256000])
256000


In [41]:
steering_on = True
run_generate(question_prompt, model, layer, coeff, steering_vectors, steering_on, sampling_kwargs, seed)

  0%|          | 0/30 [00:00<?, ?it/s]

give me only one random color and animal. Red and sheep are a classic combination, but I' Apple is not.<eos><eos>Red Apple is a classic combination, but Apple is not a Red
